In [4]:
import os

import PyPDF2
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
from PIL import Image
from pdf2image import convert_from_path

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

In [2]:
data_folder = "./data/readable"
files = os.listdir(data_folder)
print(files)

['OOP_s_pom_ Python.pdf', 'PyCharm_Профессиональная_работа_на_Python.pdf', 'Python 3. Самое необходимое. 2-е изд.pdf', 'Python. Наиболее полное руководство 2002.pdf', 'Python. Подробный справочник.pdf', 'Python. Полное руководство  (2022).pdf', 'Python. Самое необходимое.pdf', 'Python_dlya_analiza_dannyh.pdf', 'Python_Библиотеки.pdf', 'Python_Создаем_программы_и_игры.pdf', 'Алгоритмы_и_структуры_для_массивных_наборов_данных.pdf', 'Алгоритмы_принятия_решений.pdf', 'Изучаем Python (2017).pdf', 'Изучаем программирование на Python.pdf', 'Искусство _чистого_кода.pdf', 'Компьютерное_зрение-1.pdf', 'Компьютерное_зрение.pdf', 'Объекты_Стильное_ООП.pdf', 'Основы программирования в Python три в одном. Том 1 (2021).pdf', 'Программируем на Python - 2014.pdf', 'Язык программирования Python.pdf', 'Язык_программирования_С++.pdf']


# Создание csv со страницами оглавления

In [5]:
# For each file in the folder run script which asking pages of table of contents (for example: "11-14" or "6-8")
# Asking example:
# Book : 'OOP_s_pom_ Python.pdf'
# Write pages of table of contents: 

df = pd.DataFrame(columns=["book", "page_num", "text"])
for book_name in files:
    print(f"Book : '{book_name}'")
    pages = input("Write pages of table of contents: ")
    pages = pages.split("-")
    pages = list(map(int, pages))
    print(pages)
    # Extracting text from the pages of table of contents
    for page in range(pages[0], pages[1] + 1):
        text = ""
        text += extract_text(f"{data_folder}/{book_name}", page_numbers=[page-1])
        # Add to df the name of the book and text from the page
        df.loc[len(df)] = [book_name, page, text]

Book : 'OOP_s_pom_ Python.pdf'
[9, 16]
Book : 'PyCharm_Профессиональная_работа_на_Python.pdf'


ValueError: invalid literal for int() with base 10: ''

In [5]:
df.head()

,book,page_num,text
0,OOP_s_pom_ Python.pdf,11,ПОД РОБНОЕ СОД Е РЖ А НИЕ \n\nОб авторе . ....
1,OOP_s_pom_ Python.pdf,12,Область видимости и переменные экземпляра ....
2,OOP_s_pom_ Python.pdf,13,"Программы, управляемые событиями . . . . . ...."
3,OOP_s_pom_ Python.pdf,14,"Часть III. Инкапсуляция, полиморфизм \nи насле..."
4,OOP_s_pom_ Python.pdf,15,Абстрактные классы и методы . . . . . . . . ...


In [7]:
df.to_csv("books_table_of_content.csv", index=False)

In [27]:
df = pd.read_csv("books_table_of_content.csv")

In [32]:
import random

# Read some random pages from pdf to use as a sample (train data)
# For each book, read 5 random pages from the book (that are not in the table of contents)
# Save the text from these pages to a new dataframe
df_sample = pd.DataFrame(columns=["book", "page_num", "text"])
for book_name in files:
    print(f"Book : '{book_name}'")
    # Extract len of the book
    len_book = len(list(extract_pages(f"{data_folder}/{book_name}")))
    # Extracting text from the pages of table of contents (start from 25 page to the end of the book)
    usedPages = []
    for _ in range(50):
        while True:
            page = random.randint(25, len_book)
            if page not in usedPages:
                usedPages.append(page)
                break
        text = ""
        text += extract_text(f"{data_folder}/{book_name}", page_numbers=[page-1])
        # Add to df the name of the book and text from the page
        df_sample.loc[len(df_sample)] = [book_name, page, text]

Book : 'OOP_s_pom_ Python.pdf'
Book : 'PyCharm_Профессиональная_работа_на_Python.pdf'
Book : 'Python 3. Самое необходимое. 2-е изд.pdf'
Book : 'Python. Наиболее полное руководство 2002.pdf'
Book : 'Python. Подробный справочник.pdf'
Book : 'Python. Полное руководство  (2022).pdf'
Book : 'Python. Самое необходимое.pdf'
Book : 'Python_dlya_analiza_dannyh.pdf'
Book : 'Python_Библиотеки.pdf'
Book : 'Python_Создаем_программы_и_игры.pdf'
Book : 'Алгоритмы_и_структуры_для_массивных_наборов_данных.pdf'
Book : 'Алгоритмы_принятия_решений.pdf'
Book : 'Изучаем Python (2017).pdf'
Book : 'Изучаем программирование на Python.pdf'
Book : 'Искусство _чистого_кода.pdf'
Book : 'Компьютерное_зрение-1.pdf'
Book : 'Компьютерное_зрение.pdf'
Book : 'Объекты_Стильное_ООП.pdf'
Book : 'Основы программирования в Python три в одном. Том 1 (2021).pdf'
Book : 'Программируем на Python - 2014.pdf'
Book : 'Язык программирования Python.pdf'
Book : 'Язык_программирования_С++.pdf'


In [33]:
df_sample.head()

,book,page_num,text
0,OOP_s_pom_ Python.pdf,456,полиморфизма и наследования. Каждая сцена явля...
1,OOP_s_pom_ Python.pdf,91,Повторный обзор класса DimmerSwitch \n\n В сле...
2,OOP_s_pom_ Python.pdf,341,Уменьшение количества ссылок\n\nДля уменьшения...
3,OOP_s_pom_ Python.pdf,124,После сбора информации от пользователя для ope...
4,OOP_s_pom_ Python.pdf,336,"11\n\nУ ПРА В ЛЕ НИЕ П А М ЯТЬЮ, \nИС ПОЛЬЗУ ..."


In [34]:
# Prepare the data for the model (combine the table of contents and the sample data and add a label)
df_sample["label"] = 0
df["label"] = 1
df_Train = pd.concat([df, df_sample], ignore_index=True)
df_Train = df_Train.sample(frac=1).reset_index(drop=True)
df_Train.head()
# Print amount of rows
print(f"Amount of rows: {len(df_Train)}")

Amount of rows: 1293


In [35]:
df_Train.head()

,book,page_num,text,label
0,Программируем на Python - 2014.pdf,264,264 \n\nГлава 9. Объектно-ориентированное про...,0
1,Компьютерное_зрение.pdf,96,Часть D. Отслеживание движущихся объектов  ...,0
2,Python 3. Самое необходимое. 2-е изд.pdf,605,604\n\nПредметный указатель\n\nСтрока (прод.)...,0
3,Программируем на Python - 2014.pdf,186,186 \n\nГлава 6. Функции. Игра «Крест111ки-но...,0
4,Компьютерное_зрение-1.pdf,369,368  Адаптация домена и непрерывное обучени...,0


In [36]:
df_Train.to_csv("books_train.csv", index=False)

# Ты можешь просто начать здесь (Перед этим запусти первую строку с импортами)

In [3]:
# Split the data into train and test
df_Train = pd.read_csv("books_train.csv")
df_Train, df_Test = train_test_split(df_Train, test_size=0.2, random_state=42)

# Feature extraction

In [28]:
def process_page(page):
    text = extract_text(f"{data_folder}/{book_name}", page_numbers=[page-1])
    return [book_name, page+1, text]

def bookToDF(book_name, data_folder):
    pages = list(extract_pages(f"{data_folder}/{book_name}"))
    df = pd.DataFrame(
        range(len(pages)).parallel_apply(process_page).tolist(),
        columns=["book", "page_num", "text"]
    )
    return df

## TF-IDF

In [12]:
# Use the TF-IDF vectorizer to convert the text to a matrix of TF-IDF features
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(df_Train["text"])
X_test = vectorizer.transform(df_Test["text"])

In [13]:
# Train a Random Forest model
clf = RandomForestClassifier()
clf.fit(X_train, df_Train["label"])

RandomForestClassifier()

In [18]:
# Predict the test data
y_pred = clf.predict(X_test)

# Print the accuracy, precision, recall, and F1 score
print(f"Accuracy: {accuracy_score(df_Test['label'], y_pred)}")

# Take random page from the book and predict if it is a table of contents or not
book_name = "OOP_s_pom_ Python.pdf"
page = 12
text = extract_text(f"{data_folder}/{book_name}", page_numbers=[page-1])
print(clf.predict(vectorizer.transform([text])))

Accuracy: 0.9691119691119691
[1]


In [29]:
book_name_ = "Разработка_веб_приложения_GraphQL_с_React_Node_js_и_Neo4j.pdf"
data_folder_ = "./data/test"
bookDF = bookToDF(book_name_, data_folder_)
bookDF["prediction"] = clf.predict(vectorizer.transform(bookDF["text"]))

FileNotFoundError: [Errno 2] No such file or directory: './data/readable/Разработка_веб_приложения_GraphQL_с_React_Node_js_и_Neo4j.pdf'

In [26]:
from tqdm import tqdm

# Take (./data/test/Интерактивные_дашборды_и_приложения_с_Plotly_и_Dash.pdf) and get pages where is the table of contents
df_book = pd.DataFrame(columns=["book", "page_num", "text"])
book_name = "Разработка_веб_приложения_GraphQL_с_React_Node_js_и_Neo4j.pdf"
data_folder_ = "./data/test"
len_book = len(list(extract_pages(f"{data_folder_}/{book_name}")))
# Iterate over each page of the book
for page in tqdm(range(len_book)):
    text = ""
    text += extract_text(f"{data_folder_}/{book_name}", page_numbers=[page])
    # Add to df the name of the book and text from the page
    df_book.loc[len(df_book)] = [book_name, page, text]
    
# Use the model to predict if the page is a table of contents
df_book["label"] = clf.predict(vectorizer.transform(df_book["text"]))
df_book[df_book["label"] == 1]

100%|██████████| 264/264 [01:06<00:00,  3.95it/s]


,book,page_num,text,label
6,Разработка_веб_приложения_GraphQL_с_React_Node...,6,Оглавление\n\nПредисловие от издательства .......,1
8,Разработка_веб_приложения_GraphQL_с_React_Node...,8,Оглавление  7\n\nГлава 4. Библиотека Neo4j ...,1
9,Разработка_веб_приложения_GraphQL_с_React_Node...,9,8  Оглавление\n\nГлава 6. Клиент GraphQL .....,1
10,Разработка_веб_приложения_GraphQL_с_React_Node...,10,Оглавление  9\n\n8.2.3. Выгрузка данных в N...,1
256,Разработка_веб_приложения_GraphQL_с_React_Node...,256,узел﻿и﻿отношение﻿﻿70\nузел﻿и﻿свойство﻿﻿69\nдан...,1
257,Разработка_веб_приложения_GraphQL_с_React_Node...,257,256  Предметный﻿указатель\n\nупорядочение\n...,1
258,Разработка_веб_приложения_GraphQL_с_React_Node...,258,Category﻿метка﻿узла﻿﻿86\nCDN﻿(Content﻿Delivery...,1
259,Разработка_веб_приложения_GraphQL_с_React_Node...,259,258  Предметный﻿указатель\n\nвычисляемые﻿по...,1
260,Разработка_веб_приложения_GraphQL_с_React_Node...,260,MERGE﻿команда﻿﻿76\nmoviesByTitle﻿функция﻿разре...,1
261,Разработка_веб_приложения_GraphQL_с_React_Node...,261,260  Предметный﻿указатель\n\noffset﻿аргумен...,1


## Word2Vec

## ruBERT

In [27]:
# use ruBERT to vectorize the text
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
